In [1]:
import cv2 as cv
import numpy as np
from datetime import datetime, timedelta
import os
import tensorflow as tf

def is_valid(now, time_end, frame, save_path, curr_time):
    if now >= time_end:
        filename = f"{curr_time.replace(':', '-')}.png"
        cv.imwrite(os.path.join(save_path, filename), frame)
        return False
    return True

def preprocess_frame(frame, target_size=(200, 200)):
    """Resize and normalize the frame for model prediction."""
    resized_frame = cv.resize(frame, target_size)
    return np.expand_dims(resized_frame / 255.0, axis=0)

def draw_info(frame, center, curr_date, curr_time, predicted_name, confidence_score):
    """Draw date, time, and predicted name on the frame."""
    (text_width, _), _ = cv.getTextSize(curr_date, cv.FONT_HERSHEY_TRIPLEX, 0.75, 1)
    text_x = center[0] - (text_width // 2)
    text_y = center[1] + 20  # Adjust vertical position

    cv.putText(frame, curr_date, (text_x - 200, text_y - 200), 
               cv.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)
    cv.putText(frame, curr_time, (text_x - 200, text_y - 185), 
               cv.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)
    cv.putText(frame, f"Predicted: {predicted_name}", (text_x - 200, text_y - 170), 
               cv.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)
    cv.putText(frame, f"Confidence: {confidence_score*100:.2f}%", (text_x - 200, text_y - 155),
               cv.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)

def main():
    # Load the trained model
    try:
        model = tf.keras.models.load_model('face_recognition_model.h5')
    except OSError as e:
        print(f"Error: {e}")
        print("Ensure the model file 'face_recognition_model.h5' exists in the correct path.")
        return

    # Load class names
    try:
        with open('class_names.txt', 'r') as f:
            class_names = [line.strip() for line in f.readlines()]
    except FileNotFoundError as e:
        print(f"Error: {e}")
        print("Ensure the file 'class_names.txt' exists in the correct path.")
        return

    print("Class Names:", class_names)
    print('\nCamera ON!!')

    # Initialize camera
    cam = cv.VideoCapture(0)
    if not cam.isOpened():
        print("Error: Could not open camera.")
        return

    # Set up timing
    time_start = datetime.now()
    time_end = time_start + timedelta(seconds=6)

    # Create a directory to save captured frames
    save_directory = os.path.join("D:", "Entries", str(datetime.today().date()))
    os.makedirs(save_directory, exist_ok=True)

    # Precompute mask
    ret, frame = cam.read()
    if not ret:
        print("Error: Failed to capture initial frame.")
        return

    height, width, _ = frame.shape
    center = (width // 2, height // 2)
    mask = cv.rectangle(np.zeros((height, width), dtype='uint8'), 
                        (center[0] - 270, center[1] - 150), 
                        (center[0] + 270, center[1] + 150), 
                        255, thickness=-1)

    while True:
        ret, frame = cam.read()
        if not ret:
            print("Error: Failed to capture frame.")
            break

        # Flip the frame horizontally
        frame = cv.flip(frame, 1)

        # Get the current date and time
        now = datetime.now()
        curr_date = now.strftime('%d-%m-%y')
        curr_time = now.strftime('%H-%M-%S')

        # Check if the current time is within the allowed duration
        if not is_valid(now, time_end, frame, save_directory, curr_time):
            break

        # Apply the mask to the frame
        masked_vid = cv.bitwise_and(frame, frame, mask=mask)

        # Draw a rectangle around the masked area
        cv.rectangle(masked_vid, 
                     (center[0] - 275, center[1] - 155), 
                     (center[0] + 275, center[1] + 155), 
                     (255, 255, 255), thickness=1)

        # Preprocess the frame for prediction
        input_frame = preprocess_frame(frame)

        # Get predictions
        predictions = model.predict(input_frame)

        # Print prediction probabilities for debugging
        print("Predictions probabilities:", predictions)

        # Get the predicted class (highest probability)
        predicted_label = np.argmax(predictions)

        # Map the predicted label to the class name
        predicted_name = class_names[predicted_label] if predicted_label < len(class_names) else "Unknown"

        # Display the predicted name and confidence
        confidence_score = np.max(predictions)
        print(f"Predicted: {predicted_name}, Confidence: {confidence_score:.2f}")

        # Draw information on the frame
        draw_info(masked_vid, center, curr_date, curr_time, predicted_name, confidence_score)

        # Display the frame
        cv.imshow('Masked Video', masked_vid)

        # Exit loop on 'q' key press
        if cv.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cam.release()
    cv.destroyAllWindows()

if __name__ == "__main__":
    main()


C:\Users\DELL\anaconda3\envs\world1\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "





Class Names: ['Anju-Kushwala', 'Madhumita-Sharma', 'Pooja-Rani', 'Pratibha-Yadav', 'Priyanka-Santra', 'Roli-Raizada', 'Vandana-Sharma']

Camera ON!!
1/1 [==============================] - 1s 693ms/step
Predictions probabilities: [[1.3007087e-06 8.6046845e-02 1.0203302e-05 4.1347128e-04 9.5317224e-03
  6.4372599e-02 8.3962387e-01]]
Predicted: Vandana-Sharma, Confidence: 0.84
1/1 [==============================] - 0s 90ms/step
Predictions probabilities: [[1.2260560e-07 4.2579351e-03 6.2635883e-07 9.0449445e-05 1.1892667e-02
  1.9773300e-01 7.8602511e-01]]
Predicted: Vandana-Sharma, Confidence: 0.79
1/1 [==============================] - 0s 87ms/step
Predictions probabilities: [[2.1555561e-06 2.5515177e-03 4.7075118e-06 1.9385573e-04 1.6861192e-03
  4.0422451e-01 5.9133708e-01]]
Predicted: Vandana-Sharma, Confidence: 0.59
1/1 [==============================] - 0s 83ms/step
Predictions probabilities: [[1.3028701e-06 5.4813796e-03 2.4685585e-06 1.2292946e-04 7.6420512e-04
  2.3346061e-01